In [4]:
#control + F + dropdown new to search bar to find and replace 
import pandas as pd
import numpy as np
import xlsxwriter

#warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

Location = r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\Automation LDB\Data Source\202404\1st Cut -  ECL\Finance"
File = "Debtors Listing and Customer Balance Report as at April 2024 v2"

Isl_Cost = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Debtors Listing Islamic (Cost)', header=5) # skiprows = 2, nrows=18,  usecols= 'A:D'
Isl_Profit = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Debtors Listing Islamic (Profit', header=3) # skiprows = 2, nrows=18,  usecols= 'A:D'
Mora = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Modification MORA & R&R Apr2024', header=5) # skiprows = 2, nrows=18,  usecols= 'A:D'
Conv = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Debtors Listing Conv Apr 2024', header=2) # skiprows = 2, nrows=18,  usecols= 'A:D'
Accrued = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Accrued Interest Apr 2024', header=4) # skiprows = 2, nrows=18,  usecols= 'A:D'
Others_conv = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Other Debtors Apr 2024', header=4) # skiprows = 2, nrows=18,  usecols= 'A:D'
Others_Isl = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Other Debtors Islamic Apr 2024', header=4) # skiprows = 2, nrows=18,  usecols= 'A:D'

IIS = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='IIS Apr 2024', header=4) # skiprows = 2, nrows=18,  usecols= 'A:D'
PIS = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='PIS Apr 2024', header=4) # skiprows = 2, nrows=18,  usecols= 'A:D'


In [5]:

#-------------------------------------------------------------islamic----------------------------------------------------------


#Debtors Listing Islamic (Cost)
Isl_Cost1 = Isl_Cost.iloc[np.where(~Isl_Cost['Customer\nAccount'].isna())]

Isl_Cost1.columns = Isl_Cost1.columns.str.replace("\n", "_")
Isl_Cost1.columns = Isl_Cost1.columns.str.replace(" ", "")

Isl_Cost1.Customer_Account = Isl_Cost1.Customer_Account.astype(int)
Isl_Cost1.Disbursement = Isl_Cost1.Disbursement.astype(float)
Isl_Cost1.Cost_Payment = Isl_Cost1.Cost_Payment.astype(float)
Isl_Cost1.Balance_30thApril2024 = Isl_Cost1.Balance_30thApril2024.astype(float)

Isl_Cost2 = Isl_Cost1.fillna(0).groupby(['Company','Customer_Account'\
,'Currency'])[['Disbursement'\
,'Cost_Payment','Balance_30thApril2024']].sum().reset_index()

Isl_Cost2 = Isl_Cost2.rename(columns={'Balance_30thApril2024': 'Principal'}).fillna(0).sort_values(by=['Principal'],ascending=[True])
#Isl_Cost2['Sheet'] = 'Debtors Listing Islamic (Cost)'
Isl_Cost2['Financing_Type'] = 'Islamic'


#Debtors Listing Islamic (Profit)
Isl_Profit1 = Isl_Profit.iloc[np.where(~Isl_Profit['Customer\nAccount'].isna())]

Isl_Profit1.columns = Isl_Profit1.columns.str.replace("\n", "_")
Isl_Profit1.columns = Isl_Profit1.columns.str.replace(" ", "")

Isl_Profit1.Customer_Account = Isl_Profit1.Customer_Account.astype(int)
Isl_Profit1.Unearned_Profit = Isl_Profit1.Unearned_Profit.astype(float)
Isl_Profit1.Profit_Payment = Isl_Profit1.Profit_Payment.astype(float)
Isl_Profit1.Balance_30thApril2024 = Isl_Profit1.Balance_30thApril2024.astype(float)

Isl_Profit2 = Isl_Profit1.fillna(0).groupby(['Company','Customer_Account'\
,'Currency'])[['Unearned_Profit','Profit_Payment','Balance_30thApril2024']].sum().reset_index()

Isl_Profit2 = Isl_Profit2.rename(columns={'Balance_30thApril2024': 'Interest'}).fillna(0).sort_values(by=['Interest'],ascending=[True])
#Isl_Profit2['Sheet'] = 'Debtors Listing Islamic (Profit)'
Isl_Profit2['Financing_Type'] = 'Islamic'


#Combine Islamic Cost+Profit
A001 = Isl_Cost2.merge(Isl_Profit2,on=['Customer_Account','Company','Currency','Financing_Type'],how='outer',indicator=True)

A001 = A001.drop(columns=['_merge'])

A002 = A001.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type'])[['Disbursement'\
,'Cost_Payment','Principal','Unearned_Profit','Profit_Payment','Interest']].sum().reset_index()

NamaCompany = A001[['Company','Customer_Account']].drop_duplicates('Customer_Account', keep='first')
A003 = A002.merge(NamaCompany,on='Customer_Account',how='left')


#Modification MORA & R&R Apr2024
Mora1 = Mora.fillna(0).rename(columns={'Borrower code': 'Customer_Account',
                            'Borrower':'Company',
                            'Modification impact (RM)':'Mora',
                            'Islamic/ conventional':'Financing_Type'}).iloc[np.where((~Mora['Borrower code'].isna())&(Mora['Borrower code']!='Borrower code'))]

Mora1.columns = Mora1.columns.str.replace("\n", "_")
Mora1.columns = Mora1.columns.str.replace(" ", "")

Mora1.Customer_Account = Mora1.Customer_Account.astype(int)
Mora1.Mora = Mora1.Mora.astype(float)

Mora1.loc[Mora1.Currency.isin(['RM']),'Currency'] = 'MYR'

A004 = A003.merge(Mora1[['Customer_Account','Company','Currency','Financing_Type','SLOacceptancedate','Mora']],on=['Customer_Account','Company','Currency','Financing_Type'],how='outer',indicator=True)

NamaMora = A004[['Company','Customer_Account']].drop_duplicates('Customer_Account', keep='first')
A004 = A004.drop(['Company','_merge'],axis=1).merge(NamaMora,on='Customer_Account',how='left')

A004 = A004.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Cost_Payment','Principal','Unearned_Profit','Profit_Payment','Interest','Mora']].sum().reset_index()


#Other Debtors Islamic Apr2024
Others_Isl1 = Others_Isl.iloc[np.where(~Others_Isl.Customer.isna())].fillna(0)

Others_Isl1.columns = Others_Isl1.columns.str.replace("\n", "_")
Others_Isl1.columns = Others_Isl1.columns.str.replace(" ", "")

Others_Isl1.rename(columns={'Customer': 'Customer_Account',
                            'SearchTerm':'Company',
                            'Crcy':'Currency',
                            'Accumulatedbalance':'Other_Charges'},inplace=True)

Others_Isl1.Customer_Account = Others_Isl1.Customer_Account.astype(int)
Others_Isl1.Other_Charges = Others_Isl1.Other_Charges.astype(float)

Others_Isl1 = Others_Isl1.fillna(0).groupby(['Company','Customer_Account'])[['Other_Charges']].sum().reset_index()

Others_Isl1['Financing_Type'] = 'Islamic'

A005 = A004.merge(Others_Isl1,on=['Customer_Account','Company','Financing_Type'],how='outer',indicator=True)

NamaOther = A005[['Company','Customer_Account','Currency']].drop_duplicates('Customer_Account', keep='first')
A005 = A005.drop(['Company','Currency','_merge'],axis=1).merge(NamaOther,on='Customer_Account',how='left')

A005 = A005.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Cost_Payment','Principal','Unearned_Profit','Profit_Payment','Interest','Mora','Other_Charges']].sum().reset_index()


#PIS
PIS1 = PIS.iloc[np.where(~PIS.Customer.isna())].fillna(0)

PIS1.columns = PIS1.columns.str.replace("\n", "_")
PIS1.columns = PIS1.columns.str.replace(" ", "")

PIS1.rename(columns={'Customer': 'Customer_Account',
                            'SearchTerm':'Company',
                            'Crcy':'Currency',
                            'Accumulatedbalance':'Interest_in_Suspense'},inplace=True)

PIS1.Customer_Account = PIS1.Customer_Account.astype(int)
PIS1.Interest_in_Suspense = PIS1.Interest_in_Suspense.astype(float)

PIS1 = PIS1.fillna(0).groupby(['Company','Customer_Account'])[['Interest_in_Suspense']].sum().reset_index()

PIS1['Financing_Type'] = 'Islamic'

A006 = A005.merge(PIS1,on=['Customer_Account','Company','Financing_Type'],how='outer',indicator=True)

NamaPIS = A006[['Company','Customer_Account','Currency']].drop_duplicates('Customer_Account', keep='first')
A006 = A006.drop(['Company','Currency','_merge'],axis=1).merge(NamaPIS,on='Customer_Account',how='left')

A006 = A006.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Cost_Payment','Principal','Unearned_Profit','Profit_Payment','Interest','Mora','Other_Charges','Interest_in_Suspense']].sum().reset_index()



C:\Users\syahidhalid\AppData\Local\Temp\2\ipykernel_24956\1665433671.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Isl_Cost1.Customer_Account = Isl_Cost1.Customer_Account.astype(int)
C:\Users\syahidhalid\AppData\Local\Temp\2\ipykernel_24956\1665433671.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Isl_Cost1.Disbursement = Isl_Cost1.Disbursement.astype(float)
C:\Users\syahidhalid\AppData\Local\Temp\2\ipykernel_24956\1665433671.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy

#Conv

In [6]:
#Debtors Listing Conv Apr 2024
Conv1 = Conv.iloc[np.where(~Conv['Customer Account Number'].isna())]

Conv1.columns = Conv1.columns.str.replace("\n", "_")
Conv1.columns = Conv1.columns.str.replace(" ", "")

Conv1 = Conv1.rename(columns={'CustomerAccountNumber': 'Customer_Account',
                            'CustomerName':'Company',
                            'LoanCurrency':'Currency',
                            'ClosingPrincipal':'Principal'}).fillna(0)

Conv1.Customer_Account = Conv1.Customer_Account.astype(int)
Conv1.Principal = Conv1.Principal.astype(float)

#[['Customer_Account','Company','Currency','Disbursement','Repayment','Principal']]

Conv1 = Conv1.fillna(0).groupby(['Company','Customer_Account'\
,'Currency'])[['Disbursement'\
,'Repayment','Principal']].sum().reset_index()

Conv1['Financing_Type'] = 'Conventional'


#Accrued Interest Apr2024
Accrued1 = Accrued.iloc[np.where(~Accrued.Customer.isna())].fillna(0)

Accrued1.columns = Accrued1.columns.str.replace("\n", "_")
Accrued1.columns = Accrued1.columns.str.replace(" ", "")

Accrued1.rename(columns={'Customer': 'Customer_Account',
                            'SearchTerm':'Company',
                            'Crcy':'Currency',
                            'Accumulatedbalance':'Interest'},inplace=True)

Accrued1.Customer_Account = Accrued1.Customer_Account.astype(int)
Accrued1.Interest = Accrued1.Interest.astype(float)

Accrued1 = Accrued1.fillna(0).groupby(['Company','Customer_Account'])[['Interest']].sum().reset_index()

Accrued1['Financing_Type'] = 'Conventional'


#Combine Conv Principal+Accrued
C001 = Conv1.merge(Accrued1,on=['Customer_Account','Company','Financing_Type'],how='outer',indicator=True)

NamaConv = C001[['Company','Customer_Account','Currency']].drop_duplicates('Customer_Account', keep='first')
C002 = C001.drop(['Company','Currency','_merge'],axis=1).merge(NamaConv,on='Customer_Account',how='left')

C002 = C002.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Repayment','Principal','Interest']].sum().reset_index()

#Other Debtors Apr2024
Others_conv1 = Others_conv.iloc[np.where(~Others_conv.Customer.isna())].fillna(0)

Others_conv1.columns = Others_conv1.columns.str.replace("\n", "_")
Others_conv1.columns = Others_conv1.columns.str.replace(" ", "")

Others_conv1.rename(columns={'Customer': 'Customer_Account',
                            'SearchTerm':'Company',
                            'Crcy':'Currency',
                            'Accumulatedbalance':'Other_Charges'},inplace=True)

Others_conv1.Customer_Account = Others_conv1.Customer_Account.astype(int)
Others_conv1.Other_Charges = Others_conv1.Other_Charges.astype(float)

Others_conv1 = Others_conv1.fillna(0).groupby(['Company','Customer_Account'])[['Other_Charges']].sum().reset_index()

Others_conv1['Financing_Type'] = 'Conventional'

C003 = C002.merge(Others_conv1,on=['Customer_Account','Company','Financing_Type'],how='outer',indicator=True)

NamaOtherConv = C003[['Company','Customer_Account','Currency']].drop_duplicates('Customer_Account', keep='first')
C004 = C003.drop(['Company','Currency','_merge'],axis=1).merge(NamaOtherConv,on='Customer_Account',how='left')

C004 = C004.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Repayment','Principal','Interest','Other_Charges']].sum().reset_index()


#IIS
IIS1 = IIS.iloc[np.where(~IIS.Customer.isna())].fillna(0)

IIS1.columns = IIS1.columns.str.replace("\n", "_")
IIS1.columns = IIS1.columns.str.replace(" ", "")

IIS1.rename(columns={'Customer': 'Customer_Account',
                            'SearchTerm':'Company',
                            'Crcy':'Currency',
                            'Accumulatedbalance':'Interest_in_Suspense'},inplace=True)

IIS1.Customer_Account = IIS1.Customer_Account.astype(int)
IIS1.Interest_in_Suspense = IIS1.Interest_in_Suspense.astype(float)

IIS1 = IIS1.fillna(0).groupby(['Company','Customer_Account'])[['Interest_in_Suspense']].sum().reset_index()

IIS1['Financing_Type'] = 'Conventional'

C005 = C004.merge(IIS1,on=['Customer_Account','Company','Financing_Type'],how='outer', indicator=True)

NamaIIS = C005[['Company','Customer_Account','Currency']].drop_duplicates('Customer_Account', keep='first')
C005 = C005.drop(['Company','Currency','_merge'],axis=1).merge(NamaIIS,on='Customer_Account',how='left')

C005 = C005.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Repayment','Principal','Interest','Other_Charges','Interest_in_Suspense']].sum().reset_index()


#-------------------------------------------------combine-------------------------------------------------

C005['Cost_Payment'] = 0
C005['Unearned_Profit'] = 0
C005['Profit_Payment'] = 0
C005['Mora'] = 0

C006 = C005[['Customer_Account','Currency','Financing_Type','Company','Disbursement','Repayment','Cost_Payment',
             'Principal','Unearned_Profit','Profit_Payment','Interest','Mora','Other_Charges',
             'Interest_in_Suspense']]

A006['Repayment'] = 0

A007 = A006[['Customer_Account','Currency','Financing_Type','Company','Disbursement','Repayment','Cost_Payment',
             'Principal','Unearned_Profit','Profit_Payment','Interest','Mora','Other_Charges',
             'Interest_in_Suspense']]

#Isl_Cost1.columns = Isl_Profit1.columns = Mora1.columns = Conv1.columns
#appendR = pd.concat([Isl_Cost1,Isl_Profit1,Mora1,Conv1] )

C006.columns = A007.columns
appendR = pd.concat([C006,A007] )

NamaappendR = appendR[['Company','Customer_Account']].drop_duplicates('Customer_Account', keep='first')
appendR = appendR.drop(['Company'],axis=1).merge(NamaappendR,on='Customer_Account',how='left')

appendfinal = appendR.fillna(0).groupby(['Customer_Account'\
,'Currency','Financing_Type','Company'])[['Disbursement'\
,'Repayment','Cost_Payment','Principal','Unearned_Profit','Profit_Payment','Interest','Mora','Other_Charges','Interest_in_Suspense']].sum().reset_index()

appendfinal['Total Loans Outstanding (MYR)'] = appendfinal['Principal'] + appendfinal['Interest'] + appendfinal['Mora'] + appendfinal['Other_Charges']

In [7]:
appendfinal.head(1)

,Customer_Account,Currency,Financing_Type,Company,Disbursement,Repayment,Cost_Payment,Principal,Unearned_Profit,Profit_Payment,Interest,Mora,Other_Charges,Interest_in_Suspense,Total Loans Outstanding (MYR)
0,500010,USD,Conventional,PT BUMI INTERNATIONAL TANKERS,0.0,0.0,0.0,1.34e+07,0.0,0.0,0.0,0.0,252554.74,5.67e+06,1.37e+07


In [ ]:
appendfinal.to_excel(r'C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\Automation LDB\Data Source\202404\1. ECL\Finance\working\\Debtor Listing (python).xlsx', sheet_name='Export Worksheet', index = False)


In [ ]:

writer2 = pd.ExcelWriter(r'C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\Automation LDB\Data Source\202404\1. ECL\Finance\working\\Debtor Listing (python).xlsx',engine='xlsxwriter')

appendfinal.to_excel(writer2, sheet_name='Export Worksheet', index = False)

writer2._save()

In [ ]:
print(C005.shape)
print(A006.shape)

In [ ]:
C005.head(1)

In [ ]:
A006.head(1)

In [ ]:
A006.Customer_Account.value_counts()

In [ ]:
appendfinal.shape

In [ ]:
appendfinal.iloc[np.where(appendfinal.Customer_Account==500640)]

In [ ]:
appendfinal.Customer_Account.value_counts()

In [ ]:
#checking v1
sum(appendfinal['Total Loans Outstanding (MYR)'])

In [ ]:
#checking v1
sum(appendfinal['Total Loans Outstanding (MYR)'])

In [ ]:
#checking v2
sum(appendfinal['Total Loans Outstanding (MYR)'])